In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [11]:
import requests
from bs4 import BeautifulSoup
import sqlite3

def create_database():
    connection = sqlite3.connect('scraped_data.db')
    cursor = connection.cursor()

    # Crear la tabla si no existe
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS scraped_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            url TEXT,
            content TEXT
        )
    ''')

    connection.commit()
    return connection, cursor

def close_database(connection):
    connection.close()

def insert_data(connection, cursor, url, content):
    cursor.execute('''
        INSERT INTO scraped_data (url, content) VALUES (?, ?)
    ''', (url, content))

    connection.commit()

def scrape_text_from_url_and_store(url):
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Buscar el div con la clase 'Content'
        content_div = soup.find('div', class_='Content')

        if content_div:
            # Extraer el texto del div
            text = content_div.get_text(separator='\n')

            # Crear la conexión a la base de datos
            connection, cursor = create_database()

            # Insertar los datos en la base de datos
            insert_data(connection, cursor, url, text)

            # Cerrar la conexión a la base de datos
            close_database(connection)

            return text
        else:
            print(f"No se encontró el div con la clase 'Content' en la URL {url}")
            return None
    else:
        print(f"Error al obtener la URL {url}. Código de estado: {response.status_code}")
        return None

# Lista de URLs con el mismo formato HTML
urls = [
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/cambio-de-propietario', 
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/reposicion-de-tarjeta-de-circulacion',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/tarjeta-de-circulacion-digital',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/cambio-de-carroceria-o-motor',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/motocicleta/licencia',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/motocicleta/placas/alta-de-placas-de-motocicleta-nueva',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/motocicleta/placas/alta-de-placas-motocicleta-usada',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/motocicleta/control-vehicular-tarjeta-de-circulacion-y-otros/renovacion-o-reposicion-de-tarjeta-de-circulacion',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/motocicleta/control-vehicular-tarjeta-de-circulacion-y-otros/cambio-de-propietario-o-domicilio',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/placas/alta-de-placas-para-vehiculo-hibrido',
    'https://semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil-hibrido/alta-de-placas-para-vehiculo-hibrido-usado',
    'https://semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil-hibrido/baja-de-vehiculo-hibrido',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil-hibrido/baja-por-robo-o-siniestro',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil-hibrido/cambio-de-propietario',
    'https://semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil-hibrido/reposicion-tarjeta-de-circulacion',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/vehiculo-antiguo/placas/alta-de-placas-vehiculo-antiguo',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/vehiculo-antiguo/placas/baja-de-placas-auto-antiguo',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/vehiculo-antiguo/placas/baja-por-robo-o-siniestro',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/cambio-de-propietario',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/reposicion-de-tarjeta-de-circulacion',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/escuelas-de-manejo-registradas',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/placas/alta-de-placas-de-vehiculo-nuevo/alta-de-placas-de-auto-nuevo-traves-de-agencia',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/placas/alta-de-placas-de-vehiculo-nuevo/alta-de-placas-vehiculos-nuevos-por-persona-fisica-con-linea-de-captura-pagada',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/placas/alta-de-placas-vehiculos-usados/alta-de-placas-vehiculos-usados-de-otra-entidad-federativa-foraneos',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/placas/alta-de-placas-vehiculos-usados/alta-de-placas-de-vehiculos-usados-foraneos-con-registro-previo-en-la-ciudad-de-mexico',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/placas/alta-de-placas-vehiculos-usados/alta-de-placas-vehiculos-usados-de-la-cdmx',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/placas/alta-de-placas-vehiculos-usados/alta-de-placas-de-vehiculos-usados-foraneos-con-registro-previo-en-la-ciudad-de-mexico',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/cambio-de-propietario',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/reposicion-de-tarjeta-de-circulacion',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/personas-discapacidad/cambio-de-propietario-o-domicilio',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/personas-discapacidad/renovacion-o-reposicion-de-tarjeta-de-circulacion-para-vehiculos-habilitados-para-personas-con-discapacidad',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/taxis/expedicion-renovacion-o-reposicion-de-licencia-tipo-b-para-operador-de-taxi/licencia-digital-tipo-b',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/taxis/tramites/prorroga-para-reemplacamiento-taxi-2022-2023',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/taxis/mas-tramites/verificacion-taximetros',
    'https://www.semovi.cdmx.gob.mx/programas-de-mejora-del-transporte-publico/programa-de-mejora-de-taxi',
    'https://www.semovi.cdmx.gob.mx/programas-de-mejora-del-transporte-publico/programa-de-mejora-de-taxi/programa-de-financiamiento-al-transporte-publico-individual-sostenible',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/taxis/mas-temas-de-interes/cromatica',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/alta-de-unidades/alta-de-vehiculos-de-los-servicios-de-transporte-de-carga-publico',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/alta-de-unidades/alta-de-vehiculos-del-servicio-de-transporte-de-pasajeros-mercantil-y-privado-escolar-y-personal',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/alta-de-unidades/alta-de-vehiculos-del-servicio-de-transporte-de-pasajeros-mercantil-y-privado-especializado-ambulancia',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/alta-de-unidades/alta-de-vehiculos-del-servicio-de-transporte-de-pasajeros-mercantil-y-privado-turistico',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/alta-de-unidades/alta-de-vehiculos-del-servicio-de-transporte-de-pasajeros-privado-seguridad-privada',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/alta-de-unidades/alta-de-vehiculos-de-los-servicios-de-transporte-de-carga-mercantil-privado-y-particular',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/tarjeta-de-circulacion/reposicion-de-tarjeta-de-circulacion',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/permisos/otorgamiento-de-permiso-ocasional-de-transporte-de-carga',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/permisos/permiso-temporal-para-circular-de-transporte-de-pasajeros-mercantil-y-privado-y-de-carga-15-o-30-dias',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/permisos/permisos-para-la-portacion-de-aditamentos-en-transporte-de-pasajeros-carga-y-especializado',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/correccion-de-datos',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/revista-de-carga',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/autorizacion-de-sitios-de-transporte-de-carga',
    'https://www.semovi.cdmx.gob.mx/tramites-y-servicios/transporte-de-carga/tramites/revalidacion-de-permiso-para-sitios-de-carga'
]

for url in urls:
    text = scrape_text_from_url_and_store(url)

    # Imprimir el texto extraído (puedes realizar otras operaciones con el texto según tus necesidades)
    if text:
        print(f"Texto extraído de {url}:\n{text}\n{'='*50}\n")


Texto extraído de https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/cambio-de-propietario:







            Cambio de propietario o domicilio
        






Antes de iniciar tu trámite, considera:




Toda la documentación debe ser original, vigente, legible, sin tachaduras, ni enmendaduras.




Deberás contar con tu cuenta 
LlaveCDMX
 
habilitada para poder generar tu cita.




Es importante que consideres que la cita la debe generar la persona que acudirá a realizar el trámite.


Te invitamos a revisar
 
aquí
 
que tu vehículo no tenga adeudos, multas o infracciones.


Acudir con tu cita impresa, puntual, con documentación original completa y línea de captura pagada correspondiente al trámite; respetar los protocolos de higiene y sana distancia.




Los nuevos propietarios tendrán un plazo de 15 días hábiles para realizar el trámite de cambio de propietario de automóvil, motocicleta o remolque usados con 

In [12]:
import sqlite3

def show_data():
    connection = sqlite3.connect('scraped_data.db')
    cursor = connection.cursor()

    # Seleccionar todos los datos de la tabla
    cursor.execute('SELECT * FROM scraped_data')
    data = cursor.fetchall()

    # Mostrar los datos
    for row in data:
        print(f"ID: {row[0]}, URL: {row[1]}, Content: {row[2]}")

    connection.close()

# Llamar a la función para mostrar los datos almacenados
show_data()

ID: 1, URL: https://www.semovi.cdmx.gob.mx/tramites-y-servicios/vehiculos-particulares/automovil/control-vehicular-y-tarjeta-de-circulacion/cambio-de-propietario, Content: 






            Cambio de propietario o domicilio
        






Antes de iniciar tu trámite, considera:




Toda la documentación debe ser original, vigente, legible, sin tachaduras, ni enmendaduras.




Deberás contar con tu cuenta 
LlaveCDMX
 
habilitada para poder generar tu cita.




Es importante que consideres que la cita la debe generar la persona que acudirá a realizar el trámite.


Te invitamos a revisar
 
aquí
 
que tu vehículo no tenga adeudos, multas o infracciones.


Acudir con tu cita impresa, puntual, con documentación original completa y línea de captura pagada correspondiente al trámite; respetar los protocolos de higiene y sana distancia.




Los nuevos propietarios tendrán un plazo de 15 días hábiles para realizar el trámite de cambio de propietario de automóvil, motocicleta o remolque usados c

In [14]:
import sqlite3
import json

# Función para obtener todos los datos de la base de datos y guardarlos en un archivo JSON
def save_data_to_json():
    connection = sqlite3.connect('scraped_data.db')
    cursor = connection.cursor()

    # Seleccionar todos los datos de la tabla
    cursor.execute('SELECT * FROM scraped_data')
    data = cursor.fetchall()

    # Crear una lista de diccionarios para representar los datos en formato JSON
    json_data = []
    for row in data:
        json_data.append({
            'id': row[0],
            'url': row[1],
            'content': row[2]
        })

    # Guardar la lista de diccionarios en un archivo JSON
    with open('scraped_data.json', 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

    connection.close()

# Llamar a la función para guardar los datos en un archivo JSON
save_data_to_json()